# CHEMBL DATABASE

**IC50 value** is the quantitative value which indicates how much of a drug is required to inhibit 50% of a biological organism, here SARS CoV-2. 

For our study we will consider the **pIC50 value** which is the negative log of the IC50 value when converted to molar.

## About the project

### Obective
In this project, I am going to predict the concentration of a drug i.e. pIC50 value required for 50% inhibition of the SARS CoV-2 virus based upon the existing Descriptor information.

### Database
The common dataset of the molecules have been taken from **ChEMBL Database- https://www.ebi.ac.uk/chembl/**. It is a manually curated datasbase of all bioactive molecules having drug-like properties.

### Brief walk through
First of all, raw database contained a lot of unwanted data about all sorts of potential target organisms so I had to clean it to make it suitable for our target organism i.e. SARS CoV-2. 

After that I classified the molecules into 3 different classes such as *Active, Intermediate and Inactive* based upon its action against the virus, later removing the *Intermediate* class. 

Since the molecules are represented according to standard IUPAC notation, I had to assign each of them a unique numerical identity for machine interpretation. This is done through **SMILES notation.** For this purpose, I have taken the help of **RDkit Library.** This library is used to identify each molecule based upon their molecular weight and other chemical properties and calculate their respective pIC50 value.

For modelling, I have used **Linear Regression** and **Random Forest Regression.** Using the molecule's Descriptor information and corresponding pIC50 value these models are trained and used for our prediction.

##### NOTE: To aid in understanding, I have added my own comment lines. 

# The Project

### Importing the required libraries

In [30]:
import pandas as pd
import numpy as np

### Importing the dataset

In [31]:
chembl_file_path = 'chembl.csv'
df = pd.read_csv(chembl_file_path, delimiter=",") # shows the csv file

### Getting to know the data

In [32]:
df.head() # first five records

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_chembl_id,target_organism,target_pref_name,type,units,value
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0


In [33]:
df

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_chembl_id,target_organism,target_pref_name,type,units,value
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
...,...,...,...,...,...,...,...,...,...,...,...
353,CC(C)C[C@H](NC(=O)OC1(Cc2ccccc2)CCN(S(C)(=O)=O...,CHEMBL4208764,IC50,nM,4300.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,4.3
354,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C=O)C[C@...,CHEMBL4212620,IC50,nM,5500.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,5.5
355,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C[C@@H]...,CHEMBL4216101,IC50,nM,4100.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,4.1
356,CCOC(=O)N1CCC(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]...,CHEMBL4217568,IC50,nM,3200.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,3.2


In [34]:
df.shape

(358, 11)

In [35]:
# data cleaning: notna() removes any missing or NA values
df2 = df[df.standard_value.notna()]
df2

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_chembl_id,target_organism,target_pref_name,type,units,value
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,IC50,nM,100000.0,CHEMBL612575,SARS coronavirus,SARS coronavirus,IC50,uM,100.0
...,...,...,...,...,...,...,...,...,...,...,...
353,CC(C)C[C@H](NC(=O)OC1(Cc2ccccc2)CCN(S(C)(=O)=O...,CHEMBL4208764,IC50,nM,4300.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,4.3
354,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C=O)C[C@...,CHEMBL4212620,IC50,nM,5500.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,5.5
355,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C[C@@H]...,CHEMBL4216101,IC50,nM,4100.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,4.1
356,CCOC(=O)N1CCC(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]...,CHEMBL4217568,IC50,nM,3200.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,IC50,uM,3.2


In [36]:
df2.shape # data already cleaned so no reduction in size

(358, 11)

In [37]:
df2.describe() # some stats of the data

,standard_value,value
count,358.000000,358.000000
mean,61724.530726,64.830360
std,109989.818966,111.255937
min,30.000000,0.030000
25%,8325.000000,9.325000
50%,23500.000000,25.000000
75%,59587.500000,62.947500
max,1000000.000000,1000.000000


In [38]:
# list all the features
df2.keys()

Index(['canonical_smiles', 'molecule_chembl_id', 'standard_type',
       'standard_units', 'standard_value', 'target_chembl_id',
       'target_organism', 'target_pref_name', 'type', 'units', 'value'],
      dtype='object')

### Preparing the data

In [39]:
# 'standard_value' is the IC50 value (in nM) of the molecules
# 'value' is the corresponding value in uM
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive") # or pIC50 <= 5
  elif float(i) <= 1000:
    bioactivity_class.append("active") # or pIC50 >= 6 
  else:
    bioactivity_class.append("intermediate")

In [40]:
selection = ['canonical_smiles', 'molecule_chembl_id', 'standard_value']
df3 = df2[selection]
df3

,canonical_smiles,molecule_chembl_id,standard_value
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,100000.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,100000.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,100000.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,100000.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,100000.0
...,...,...,...
353,CC(C)C[C@H](NC(=O)OC1(Cc2ccccc2)CCN(S(C)(=O)=O...,CHEMBL4208764,4300.0
354,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C=O)C[C@...,CHEMBL4212620,5500.0
355,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C[C@@H]...,CHEMBL4216101,4100.0
356,CCOC(=O)N1CCC(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]...,CHEMBL4217568,3200.0


In [41]:
# pd.Series converts the list into a one-dimensional array with name 'bioactivity_class'
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
# axis = 0 is along row, axis = 1 is along column
df4 = pd.concat([df3, bioactivity_class], axis=1) # concat along column
df4

,canonical_smiles,molecule_chembl_id,standard_value,bioactivity_class
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,100000.0,inactive
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,100000.0,inactive
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,100000.0,inactive
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,100000.0,inactive
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,100000.0,inactive
...,...,...,...,...
353,CC(C)C[C@H](NC(=O)OC1(Cc2ccccc2)CCN(S(C)(=O)=O...,CHEMBL4208764,4300.0,intermediate
354,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C=O)C[C@...,CHEMBL4212620,5500.0,intermediate
355,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C[C@@H]...,CHEMBL4216101,4100.0,intermediate
356,CCOC(=O)N1CCC(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]...,CHEMBL4217568,3200.0,intermediate


#### Importing RDKit library

In [42]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

#### This function below takes the canonical SMILES one-by-one, extracts the required info i.e Mol Wt, LogP (Partition Coefficient), HDonors, HAcceptors etc and returns them into a dataframe.

In [43]:
# blank lines are interpreted as 'end' of function
# verbose = 'False' helps make the code readable
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1) # create an ndarray, baseData = [] is a list
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol) # molecular weight in g/mol
        desc_MolLogP = Descriptors.MolLogP(mol) # logP value of molecule
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
        
        # appending into baseData
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [44]:
df_lipinski = lipinski(df4.canonical_smiles)

In [45]:
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,240.219,-2.4278,4.0,7.0
1,240.219,-2.4278,4.0,7.0
2,239.231,-1.8228,4.0,6.0
3,278.268,-1.0166,6.0,7.0
4,240.215,-2.1991,5.0,5.0
...,...,...,...,...
353,668.767,-2.9572,4.0,10.0
354,524.659,2.5169,3.0,7.0
355,628.721,-1.8144,4.0,10.0
356,468.551,0.9581,3.0,7.0


In [46]:
df_combined = pd.concat([df4,df_lipinski], axis=1)

In [47]:
df_combined
# IC50 -> 'inactive, active and intermediate' -> value info

,canonical_smiles,molecule_chembl_id,standard_value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,100000.0,inactive,240.219,-2.4278,4.0,7.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,100000.0,inactive,240.219,-2.4278,4.0,7.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,100000.0,inactive,239.231,-1.8228,4.0,6.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,100000.0,inactive,278.268,-1.0166,6.0,7.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,100000.0,inactive,240.215,-2.1991,5.0,5.0
...,...,...,...,...,...,...,...,...
353,CC(C)C[C@H](NC(=O)OC1(Cc2ccccc2)CCN(S(C)(=O)=O...,CHEMBL4208764,4300.0,intermediate,668.767,-2.9572,4.0,10.0
354,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C=O)C[C@...,CHEMBL4212620,5500.0,intermediate,524.659,2.5169,3.0,7.0
355,CCC1(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C[C@@H]...,CHEMBL4216101,4100.0,intermediate,628.721,-1.8144,4.0,10.0
356,CCOC(=O)N1CCC(OC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]...,CHEMBL4217568,3200.0,intermediate,468.551,0.9581,3.0,7.0


In [48]:
# pIC50 value
import numpy as np

def pIC50(col):
    pIC50 = []

    for i in col['standard_value']:
        molar = i*(10**-9) # nM to M
        pIC50.append(-np.log10(molar))

    col['pIC50'] = pIC50
    x = col.drop('standard_value', 1) # drops the 'standard_value' column
        
    return x

In [49]:
df_final = pIC50(df_combined)
df_final.head()

,canonical_smiles,molecule_chembl_id,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,inactive,240.219,-2.4278,4.0,7.0,4.0
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,inactive,240.219,-2.4278,4.0,7.0,4.0
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,inactive,239.231,-1.8228,4.0,6.0,4.0
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,inactive,278.268,-1.0166,6.0,7.0,4.0
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,inactive,240.215,-2.1991,5.0,5.0,4.0


In [50]:
# remove the intermediate bioactivity_class
df_2class = df_final[df_final.bioactivity_class != 'intermediate']
df_2class

,canonical_smiles,molecule_chembl_id,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,CHEMBL204499,inactive,240.219,-2.4278,4.0,7.0,4.000000
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,CHEMBL203308,inactive,240.219,-2.4278,4.0,7.0,4.000000
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,CHEMBL381539,inactive,239.231,-1.8228,4.0,6.0,4.000000
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,CHEMBL225045,inactive,278.268,-1.0166,6.0,7.0,4.000000
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,CHEMBL224363,inactive,240.215,-2.1991,5.0,5.0,4.000000
...,...,...,...,...,...,...,...,...
342,CC(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CO)C(=O)N[C...,CHEMBL3818400,inactive,604.614,-4.3257,10.0,10.0,5.000000
343,CC(=O)N[C@@H](Cc1cnc[nH]1)C(=O)N[C@@H](CO)C(=O...,CHEMBL3818761,inactive,626.668,-3.8346,10.0,10.0,4.301030
345,CC(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CO)C(=O)N[C...,CHEMBL3818028,inactive,605.558,-6.4964,11.0,11.0,4.698970
348,CC(C)C[C@H](NC(=O)OC1CCN(S(C)(=O)=O)CC1)C(=O)N...,CHEMBL4209146,inactive,474.580,-0.2388,3.0,7.0,4.540608


In [51]:
df_final.shape

(358, 8)

In [52]:
# 72 molecules removed
df_2class.shape

(286, 8)

In [53]:
df_final.bioactivity_class.unique() # unique values of bioactivity class

array(['inactive', 'active', 'intermediate'], dtype=object)

In [54]:
df_2class.bioactivity_class.unique()

array(['inactive', 'active'], dtype=object)

#### Using 'MORDRED DESCRIPTOR' we get the descriptor information stored in *descriptors.csv* file

In [55]:
# reading the 'descriptors.csv'
df = pd.read_csv('descriptors.csv')

In [56]:
df

,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,12.963281,12.248253,0,0,21.174968,2.457054,4.666147,21.174968,1.245586,...,9.696279,65.758522,240.085855,8.278823,515,25,88.0,105.0,7.027778,3.805556
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,12.963281,12.248253,0,0,21.174968,2.457054,4.666147,21.174968,1.245586,...,9.696279,65.758522,240.085855,8.278823,515,25,88.0,105.0,7.027778,3.805556
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,12.963281,12.248253,0,0,21.174968,2.457054,4.666147,21.174968,1.245586,...,9.696279,65.758522,239.090606,7.969687,515,25,88.0,105.0,7.027778,3.805556
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,15.710828,14.057406,0,0,25.397628,2.520227,4.881224,25.397628,1.269881,...,10.082679,70.336447,278.101505,8.179456,753,34,110.0,135.0,7.500000,4.361111
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,12.963281,11.871195,0,0,21.225622,2.457803,4.799537,21.225622,1.248566,...,9.773208,64.168575,240.074621,8.278435,496,27,88.0,105.0,7.027778,3.805556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,CC(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CO)C(=O)N[C...,30.498974,27.458313,2,0,47.980635,2.337951,4.675903,47.980635,1.142396,...,10.220595,78.169497,604.270421,7.369151,6806,59,192.0,210.0,21.166667,9.583333
282,CC(=O)N[C@@H](Cc1cnc[nH]1)C(=O)N[C@@H](CO)C(=O...,32.401515,28.703440,1,0,51.855513,2.341227,4.679766,51.855513,1.178534,...,10.295834,93.631431,626.302390,7.282586,7737,61,206.0,228.0,20.166667,10.000000
283,CC(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CO)C(=O)N[C...,30.498974,27.458313,2,0,47.980635,2.337951,4.675903,47.980635,1.142396,...,10.220595,78.169497,605.229284,7.860121,6806,59,192.0,210.0,21.166667,9.583333
284,CC(C)C[C@H](NC(=O)OC1CCN(S(C)(=O)=O)CC1)C(=O)N...,24.215739,20.437222,0,0,38.259791,2.338938,4.675729,38.259791,1.195618,...,10.135670,80.939679,474.214820,7.185073,3357,44,158.0,177.0,13.312500,7.055556


### Preprocessing the descriptor data

The collected descriptor data has been spread unevenly. In order to work with it we need to scale it to proper size.

In [57]:
from sklearn.preprocessing import scale

# dropping the missing value along rows and columns
df1 = df.dropna(axis=1)
df2 = df1.dropna(axis=0)
df2

,name,ABC,ABCGG,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,NC(=O)c1ncn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)n1,12.963281,12.248253,0,0,5,5,29,17,0,...,7.515889,9.696279,65.758522,240.085855,8.278823,515,25,88.0,105.0,3.805556
1,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)nn1,12.963281,12.248253,0,0,5,5,29,17,0,...,7.515889,9.696279,65.758522,240.085855,8.278823,515,25,88.0,105.0,3.805556
2,NC(=O)c1cn([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)cn1,12.963281,12.248253,0,0,5,5,30,17,0,...,7.515889,9.696279,65.758522,239.090606,7.969687,515,25,88.0,105.0,3.805556
3,Nc1nc(O)c2[nH]cc([C@@H]3C=C(CO)[C@@H](O)[C@H]3...,15.710828,14.057406,0,0,9,10,34,20,0,...,7.627057,10.082679,70.336447,278.101505,8.179456,753,34,110.0,135.0,4.361111
4,O=c1[nH]cc([C@@H]2C=C(CO)[C@@H](O)[C@H]2O)c(=O...,12.963281,11.871195,0,0,6,6,29,17,0,...,7.002156,9.773208,64.168575,240.074621,8.278435,496,27,88.0,105.0,3.805556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,CC(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CO)C(=O)N[C...,30.498974,27.458313,2,0,0,0,82,42,0,...,0.000000,10.220595,78.169497,604.270421,7.369151,6806,59,192.0,210.0,9.583333
282,CC(=O)N[C@@H](Cc1cnc[nH]1)C(=O)N[C@@H](CO)C(=O...,32.401515,28.703440,1,0,5,5,86,44,0,...,6.259581,10.295834,93.631431,626.302390,7.282586,7737,61,206.0,228.0,10.000000
283,CC(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CO)C(=O)N[C...,30.498974,27.458313,2,0,0,0,77,42,0,...,0.000000,10.220595,78.169497,605.229284,7.860121,6806,59,192.0,210.0,9.583333
284,CC(C)C[C@H](NC(=O)OC1CCN(S(C)(=O)=O)CC1)C(=O)N...,24.215739,20.437222,0,0,0,0,66,32,0,...,6.529419,10.135670,80.939679,474.214820,7.185073,3357,44,158.0,177.0,7.055556


#### Removing the far spread data values along both sides

In [58]:
# scaling the descriptors
threshold = 1000

df3=df2.drop(df2.mean()[df2.mean()>threshold].index.values, axis=1) # drops columns with mean>1000
df4=df3.drop(df3.mean()[df3.mean()==0].index.values, axis=1) 
df5=df4.drop('name',axis=1)
df5.describe()

,ABC,ABCGG,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nHetero,nH,...,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPol,Zagreb1,Zagreb2,mZagreb2
count,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,...,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,20.919724,16.859857,0.185315,0.202797,13.276224,13.604895,45.807692,26.944056,7.545455,18.863636,...,8.517530,4.439254,10.176833,71.187004,386.356530,8.751683,42.898601,141.237762,166.356643,5.899233
std,6.008200,4.501045,0.440009,1.060019,6.578321,6.816480,17.095236,7.924447,3.278269,10.514431,...,0.317805,3.429485,0.356448,11.525443,110.309655,1.565157,13.791691,40.045844,47.354787,1.781224
min,6.692130,6.355891,0.000000,0.000000,0.000000,0.000000,15.000000,9.000000,1.000000,4.000000,...,7.181592,0.000000,8.760767,37.736937,126.031694,5.468855,9.000000,42.000000,45.000000,2.000000
25%,16.674162,13.803291,0.000000,0.000000,10.000000,10.000000,34.250000,21.000000,5.000000,12.000000,...,8.299514,0.000000,9.920509,62.158094,298.586405,7.938425,33.000000,112.000000,130.250000,4.569444
50%,18.559325,15.302403,0.000000,0.000000,12.000000,12.000000,43.000000,24.000000,7.000000,16.000000,...,8.576782,6.605298,10.226585,70.931869,362.029197,8.663819,42.000000,130.000000,157.000000,5.361111
75%,24.973986,19.251582,0.000000,0.000000,17.000000,17.000000,51.750000,32.000000,9.000000,20.750000,...,8.745284,7.195187,10.422817,78.996100,456.765513,9.521265,52.000000,168.000000,197.000000,7.020833
max,43.770640,31.893948,2.000000,10.000000,36.000000,36.000000,104.000000,54.000000,20.000000,58.000000,...,9.460866,8.328693,11.188897,101.247585,742.080614,14.960204,104.000000,308.000000,375.000000,11.944444


#### Scaling the data such that the mean is 0 and variance is 1, this is done so that we get a standardised data in the end

In [59]:
df4_scaled=scale(df5) 
df4_scaled_pd=pd.DataFrame(df4_scaled)
df_X = df4_scaled_pd
df_X

,0,1,2,3,4,5,6,7,8,9,...,827,828,829,830,831,832,833,834,835,836
0,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.984903,-1.257058,0.138897,-0.653927,...,-1.442468,0.898685,-1.350538,-0.471825,-1.328325,-0.302646,-1.300056,-1.331751,-1.297951,-1.177475
1,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.984903,-1.257058,0.138897,-0.653927,...,-1.442468,0.898685,-1.350538,-0.471825,-1.328325,-0.302646,-1.300056,-1.331751,-1.297951,-1.177475
2,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.926305,-1.257058,-0.166677,-0.558653,...,-1.442468,0.898685,-1.350538,-0.471825,-1.337363,-0.500504,-1.300056,-1.331751,-1.297951,-1.177475
3,-0.868484,-0.623714,-0.421899,-0.191650,-0.651187,-0.529777,-0.691911,-0.877819,0.138897,-0.463379,...,-0.414769,0.931158,-0.264608,-0.073928,-0.983094,-0.366244,-0.646346,-0.781417,-0.663325,-0.865033
4,-1.326585,-1.110277,-0.421899,-0.191650,-1.108030,-1.117619,-0.984903,-1.257058,-0.166677,-0.653927,...,-1.294610,0.748624,-1.134338,-0.610018,-1.328427,-0.302894,-1.154787,-1.331751,-1.297951,-1.177475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,1.597157,2.358793,4.131427,-0.191650,-2.021716,-1.999381,2.120810,1.903267,3.194634,2.013748,...,0.461878,-1.296706,0.122987,0.606895,1.978937,-0.884866,1.169517,1.269825,0.923240,2.071923
282,1.914370,2.635909,1.854764,-0.191650,-1.260311,-1.264579,2.355203,2.156092,3.194634,2.204296,...,0.695746,0.531718,0.334437,1.950794,2.179016,-0.940271,1.314786,1.620037,1.304016,2.306254
283,1.597157,2.358793,4.131427,-0.191650,-2.021716,-1.999381,1.827818,1.903267,3.805781,1.537377,...,0.461878,-1.296706,0.122987,0.606895,1.987645,-0.570630,1.169517,1.269825,0.923240,2.071923
284,0.549548,0.796179,-0.421899,-0.191650,-2.021716,-1.999381,1.183236,0.639137,1.361192,1.442103,...,0.100058,0.610537,-0.115683,0.847670,0.797866,-1.002683,0.080000,0.419310,0.225152,0.650311


In [60]:
df_Y = df_2class['pIC50']
df_Y.reset_index(drop=True, inplace=True)
df_Y

0      4.000000
1      4.000000
2      4.000000
3      4.000000
4      4.000000
         ...   
281    5.000000
282    4.301030
283    4.698970
284    4.540608
285    4.375718
Name: pIC50, Length: 286, dtype: float64

### Modelling

In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model

In [62]:
X = df_X
X

,0,1,2,3,4,5,6,7,8,9,...,827,828,829,830,831,832,833,834,835,836
0,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.984903,-1.257058,0.138897,-0.653927,...,-1.442468,0.898685,-1.350538,-0.471825,-1.328325,-0.302646,-1.300056,-1.331751,-1.297951,-1.177475
1,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.984903,-1.257058,0.138897,-0.653927,...,-1.442468,0.898685,-1.350538,-0.471825,-1.328325,-0.302646,-1.300056,-1.331751,-1.297951,-1.177475
2,-1.326585,-1.026359,-0.421899,-0.191650,-1.260311,-1.264579,-0.926305,-1.257058,-0.166677,-0.558653,...,-1.442468,0.898685,-1.350538,-0.471825,-1.337363,-0.500504,-1.300056,-1.331751,-1.297951,-1.177475
3,-0.868484,-0.623714,-0.421899,-0.191650,-0.651187,-0.529777,-0.691911,-0.877819,0.138897,-0.463379,...,-0.414769,0.931158,-0.264608,-0.073928,-0.983094,-0.366244,-0.646346,-0.781417,-0.663325,-0.865033
4,-1.326585,-1.110277,-0.421899,-0.191650,-1.108030,-1.117619,-0.984903,-1.257058,-0.166677,-0.653927,...,-1.294610,0.748624,-1.134338,-0.610018,-1.328427,-0.302894,-1.154787,-1.331751,-1.297951,-1.177475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,1.597157,2.358793,4.131427,-0.191650,-2.021716,-1.999381,2.120810,1.903267,3.194634,2.013748,...,0.461878,-1.296706,0.122987,0.606895,1.978937,-0.884866,1.169517,1.269825,0.923240,2.071923
282,1.914370,2.635909,1.854764,-0.191650,-1.260311,-1.264579,2.355203,2.156092,3.194634,2.204296,...,0.695746,0.531718,0.334437,1.950794,2.179016,-0.940271,1.314786,1.620037,1.304016,2.306254
283,1.597157,2.358793,4.131427,-0.191650,-2.021716,-1.999381,1.827818,1.903267,3.805781,1.537377,...,0.461878,-1.296706,0.122987,0.606895,1.987645,-0.570630,1.169517,1.269825,0.923240,2.071923
284,0.549548,0.796179,-0.421899,-0.191650,-2.021716,-1.999381,1.183236,0.639137,1.361192,1.442103,...,0.100058,0.610537,-0.115683,0.847670,0.797866,-1.002683,0.080000,0.419310,0.225152,0.650311


In [63]:
y = df_Y
y

0      4.000000
1      4.000000
2      4.000000
3      4.000000
4      4.000000
         ...   
281    5.000000
282    4.301030
283    4.698970
284    4.540608
285    4.375718
Name: pIC50, Length: 286, dtype: float64

### Train Test Split 

#### 4:1 ratio i.e. 80% Training Data and 20% Test Data

This is done so that 80% of the data will be used for training the model and rest 20% will be used for testing it. This *Test Data* is also used to calculate the accuracy of the model.

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [65]:
X_train.shape, y_train.shape # 80%

((228, 837), (228,))

In [66]:
X_test.shape, y_test.shape # 20%

((58, 837), (58,))

### The Models

In [70]:
# Linear Regression

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = pd.DataFrame(lr.predict(X_test))
print('\nPredicted values of Linear Regression model: \n', y_pred_lr)
print('\nTest values of Linear Regression model: \n', y_test)


Predicted values of Linear Regression model: 
            0
0   4.744727
1   3.625906
2   3.395752
3   1.557100
4   2.243981
5   4.744727
6   2.955768
7   3.398230
8   3.511938
9   4.877947
10  3.250053
11  4.744727
12  4.008085
13  3.950007
14  4.358802
15  5.610953
16  1.792158
17  4.412289
18  1.557100
19  3.766309
20  7.415400
21  4.127844
22  3.713736
23  4.412289
24  2.717316
25  3.524176
26  2.772733
27  6.549661
28  4.900921
29  4.732855
30  5.121331
31  7.347311
32  5.372527
33  3.530342
34  2.996884
35  3.698970
36  4.956046
37  4.469138
38  2.637023
39  4.390227
40  4.208309
41  4.124939
42  3.867083
43  2.637023
44  4.301030
45  5.892146
46  6.417460
47  4.866461
48  3.471102
49  4.166303
50  6.093911
51  4.423967
52  4.150483
53  5.878876
54  3.346937
55  6.093911
56  6.704276
57  4.932231

Test values of Linear Regression model: 
 111    4.970616
43     4.154902
166    4.397940
202    3.751781
24     6.853872
105    4.605548
265    4.301030
5      4.000000
36     6.00877

In [71]:
# Random Forest Regression

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

y_pred_rf = pd.DataFrame(model.predict(X_test))
print('\nPredicted values of Random Forest Regression model: \n', y_pred_rf)
print('\nTest values of Random Forest Regression model: \n', y_test)


Predicted values of Random Forest Regression model: 
            0
0   4.685966
1   3.665746
2   5.237870
3   4.215511
4   4.401541
5   4.685966
6   4.323651
7   4.087599
8   5.150905
9   4.756799
10  5.124075
11  4.685966
12  4.572570
13  4.047556
14  3.866439
15  3.981476
16  4.531882
17  4.557159
18  4.215511
19  4.445510
20  6.274596
21  4.292089
22  4.074581
23  4.557159
24  4.156790
25  4.308352
26  4.066088
27  6.686365
28  4.639819
29  4.617093
30  4.516943
31  6.298554
32  4.478055
33  4.054251
34  5.164129
35  4.044260
36  4.740834
37  4.699884
38  4.470003
39  4.471928
40  4.261006
41  4.186628
42  3.970043
43  4.470003
44  4.321403
45  4.748421
46  6.805068
47  4.705322
48  4.456716
49  4.569104
50  4.590248
51  4.646174
52  4.366459
53  5.369382
54  4.862493
55  4.590248
56  5.435225
57  4.373143

Test values of Random Forest Regression model: 
 111    4.970616
43     4.154902
166    4.397940
202    3.751781
24     6.853872
105    4.605548
265    4.301030
5      4.000000


### Evaluation of models

#### **R2 score** gives the information about the goodness of fit of a model. Ideal value is 1.

In [72]:
r2 = lr.score(X_test, y_test)
print('R2 score of Linear Regression: {}'.format(r2))

r2 = model.score(X_test, y_test)
print('R2 score of Random Forest Regression: {:.3f}'.format(r2))

ms_lr = mean_squared_error(y_test, y_pred_lr)
ms_rf = mean_squared_error(y_test, y_pred_rf)

print("Mean squared error of Linear Regression model: {}".format(ms_lr))
print("Mean squared error of Random Forest Regression model: {}".format(ms_rf))

R2 score of Linear Regression: -0.9108920980611555
R2 score of Random Forest Regression: 0.525
Mean squared error of Linear Regression model: 1.7970572829574463
Mean squared error of Random Forest Regression model: 0.44685433024611493


# Take home message

Although my project was based on *Drug Discovery* which is altogether a different area of research. From this my main aim was to convey an insight into working with real life Machine Learning research problem.

As we all saw, any ML problem is 90% of data analysis and pre-processing and the modelling and prediction contributes merely 10% of it.

Therefore it is very much important to work with **good** and **large** amount of datasets which not only saves time but also helps get better results.

## Use in Astrophysics

As many Telescope facilities have come online, the amount of data produced has increased manifolds (~20 trillion bytes of raw data per night of observation) as a result of which there has been a significant rise in the use of AI and ML in the field of Astrophysics.

### AstroPy

<a href="https://www.aanda.org/articles/aa/full_html/2013/10/aa22068-13/aa22068-13.html"><h2>Astropy: A community Python package for astronomy</h2></a>

This paper describes about astroPy, a robust library for astronomical research, data processing, and data analysis.

Contains key functionalities and common tools needed for performing astronomy and astrophysics with Python. 

### AstroML

<a href="https://ieeexplore.ieee.org/document/6382200"><h2>Introduction to astroML: Machine learning for astrophysics</h2></a>

This paper intorduces to AstroML, a Python module for machine learning and data mining in Astrophysics.

Built on numpy, scipy, scikit-learn, matplotlib, and astropy.

Contains a growing library of statistical and machine learning routines for analyzing astronomical data in Python, loaders for several open astronomical datasets, and a large suite of examples of analyzing and visualizing astronomical datasets.
    